In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
import configparser
import os
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from numpy.linalg import norm
from scipy.spatial.distance import pdist
import plotly_express as px
import seaborn as sns; sns.set()
import configparser
config = configparser.ConfigParser()
config.read("../../../env-sample.ini")
data_home = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data'
output_dir = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/output'
local_lib = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/DS5001-2025-01-R/lessons/lib'
import sys
sys.path.append(local_lib)
from textparser import TextParser

clip_pats = [
    r"\*\*\*\s*START OF",
    r"\*\*\*\s*END OF"
]

# All are 'chap'and 'm'
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
ohco_pat_list = [
    (5200,   rf"^\s*CHAPTER\s+{roman}\s*$"), #Metamorphosis
    (7849,   rf"^\s*{roman}\s*$"), #The Trial
    (6969,  rf"^\s*LETTER .* to .*$"), # The Castle
    (6262,   rf"^CHAPTER\s+{roman}$"), # Amerika
    (6161,   rf"^CHAPTER\s+\d+$"), # The Judgement
    (6060,   rf"^Chapter\s+\d+$"), # Dearest Father
    (6363,  rf"^Chapter\s+\d+$"), # In the Penal colony
    (6464,   rf"^CHAPTER\s+\d+$"), # The Hunger Artist
    (6565, rf"^\s*CHAPTER\s+{roman}\."), # The Jackals and Arabs
    (6666, rf"^\s*CHAPTER\s+{roman}\s*$"), # A Country Doctor
    (6767, rf"^\s*CHAPTER\s+{roman}\s*$"), # An Imperial Message
    (5959,  rf"^(?:ETYMOLOGY|EXTRACTS|CHAPTER)"), # A report for an Academy
    (5858,  rf"^\s*CHAPTER\s+{roman}\.\s*$"), # The Great Wall of China
    (5757, rf"^\s*{roman}\.\s*$"), # The Hunter Gracchus
    (5656,  rf"^\s*{roman}\. .*$"), # Up in the Gallery
    (5555, rf"^CHAPTER\s+{roman}\.?$"), # Before the Law
    (5454, rf"^\s*[A-Z,;-]+\.\s*$"), # Josephine the Songstress
    (5353,  rf"^CHAPTER "), # The Burrow
    (5252, rf"^CHAPTER\s+{roman}\.\s*$"), # Blumfeld
    (23532, rf"Chapter\s+{roman}") # Meditation
]
chapter_regexes = [
    (5200,   rf"^\s*{roman}\s*$"),
    (7849,   rf"^\s*Chapter\s+(?:One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten)\s*$"),
    (6969,   rf"^\s*\d+\s*$"),
    (6262,   rf"^\s*\d+\s*$"),
    (6161,   "NOCHAPTERS"),
    (6060,   "NOCHAPTERS"),
    (6363,   "NOCHAPTERS"),
    (6464,   "NOCHAPTERS"),
    (6565,   "NOCHAPTERS"),
    (6666,   "NOCHAPTERS"),
    (6767,   "NOCHAPTERS"),
    (5959,   "NOCHAPTERS"),
    (5858,   "NOCHAPTERS"),
    (5757,   "NOCHAPTERS"),
    (5656,   "NOCHAPTERS"),
    (5555,   "NOCHAPTERS"),
    (5454,   "NOCHAPTERS"),
    (5353,   "NOCHAPTERS"),
    (5252,   "NOCHAPTERS"),
    (23532,  rf"^(Children on the country road|Unmasking a con artist|The Sudden Walk|Resolutions|The trip to the mountains|The Bachelor's Misfortune|The Merchant|Distracted Looking Out|The Way Home|The Passers-by|Passenger|Dresses|The rejection|Food for thought for gentlemen riders|The Alley Window|Desire to become an Indian|The Trees|Unhappiness)$")  # Poem title on line 1
]
ohco_pat_list = chapter_regexes
source_files = f'{data_home}'
source_file_list = sorted(glob(f"{source_files}/*.*"))

book_data = []
for source_file_path in source_file_list:
    # Get the filename only, e.g. 'pg5353.txt'
    filename = os.path.basename(source_file_path)
    # Extract the numeric ID from the filename (remove 'pg' and '.txt')
    book_id = int(filename.replace('pg', '').replace('.txt', ''))
    # Use filename (without extension) as a raw title (optional: clean further)
    book_title = filename.replace('.txt', '').replace('_', ' ')
    # Append a tuple of (book_id, path, title)
    book_data.append((book_id, source_file_path, book_title))
# Convert to DataFrame
LIB = pd.DataFrame(book_data, columns=['book_id', 'source_file_path', 'raw_title']) \
        .set_index('book_id') \
        .sort_index()
book_titles = {
    5200: "Metamorphosis",
    7849: "The Trial",
    6969: "The Castle",
    6262: "Amerika",
    6161: "The Judgement",
    6060: "Dearest Father",
    6363: "In the Penal Colony",
    6464: "The Hunger Artist",
    6565: "The Jackals and Arabs",
    6666: "A Country Doctor",
    6767: "An Imperial Message",
    5959: "A Report for an Academy",
    5858: "The Great Wall of China",
    5757: "The Hunter Gracchus",
    5656: "Up in the Gallery",
    5555: "Before the Law",
    5454: "Josephine the Songstress",
    5353: "The Burrow",
    5252: "Blumfeld",
    23532: "Meditation"
}
book_titles = {f'pg{key}': value for key, value in book_titles.items()}
try:
    LIB['author'] = 'KAFKA, FRANZ'
    LIB['title'] = LIB.raw_title.replace(book_titles).str.upper()
    LIB = LIB.drop('raw_title', axis=1)
except AttributeError:
    pass
LIB['chap_regex'] = LIB.index.map(pd.Series({x[0]:x[1] for x in ohco_pat_list}))
LIB
# This cell takes 16 seconds to run
def tokenize_collection(LIB):

    clip_pats = [
        r"\*\*\*\s*START OF",
        r"\*\*\*\s*END OF"
    ]

    books = []
    for book_id in LIB.index:

        # Announce
        print("Tokenizing", book_id, LIB.loc[book_id].title)

        # Define vars
        chap_regex = LIB.loc[book_id].chap_regex
        ohco_pats = [('chap', chap_regex, 'm')]
        src_file_path = LIB.loc[book_id].source_file_path

        # Create object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)
        # text = TextImporter(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats) 

        # Define parameters
        text.verbose = True
        text.strip_hyphens = True
        text.strip_whitespace = True

        # Parse
        text.import_source().parse_tokens();

        # Name things
        text.TOKENS['book_id'] = book_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)

        # Add to list
        books.append(text.TOKENS)
        
    # Combine into a single dataframe
    CORPUS = pd.concat(books).sort_index()

    # Clean up
    del(books)
    del(text)
        
    print("Done")
        
    return CORPUS
CORPUS = tokenize_collection(LIB)

CORPUS = CORPUS[CORPUS.term_str != '']
CORPUS['pos_group'] = CORPUS.pos.str[:2]
CORPUS
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['n_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack().count(1)
VOCAB['cat_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos_group.apply(lambda x: set(x))
VOCAB['n_pos'] = CORPUS[['term_str','pos']].value_counts().unstack().count(1)
VOCAB['cat_pos'] = CORPUS[['term_str','pos']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos.apply(lambda x: set(x))
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['porter_stem'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.apply(lambda x: stemmer2.stem(x.name), 1)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.apply(lambda x: stemmer3.stem(x.name), 1)

VOCAB
# Functions to create TFIDF:
# Bring into your notebook the functions you created previously to generate a BOW table and compute TFIDF values. 
# Extend the TFIDF function so that it also returns the DFIDF value for each term in the VOCAB.
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

def BOW(tokendf, ocholevel):
    return tokendf.groupby(bags[ocholevel]+['term_str']).term_str.count().to_frame('n') 

def BOWtoTFIDF(BOW, tf_method, CORPUS, ocholevel = 2):
    
    # I added another parameter for the CORPUS because otherwise it becomes really tedious to undo the BOW function

    DTCM = BOW.n.unstack(fill_value=0)
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log2(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T
    DF = DTCM.astype('bool').sum() 
    N = DTCM.shape[0]
    IDF = IDF = np.log2(N / DF)
    TFIDF = TF * IDF
    # Extend the TFIDF function so that it also returns the DFIDF value for each term in the VOCAB.

    # We can adjust ocho level here if we want to, default is 2
    DOC = gather_docs(CORPUS, 2)
    DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))
    ngram_range = (1,2)
    n_terms = 4000
    count_engine = CountVectorizer(
        stop_words = 'english',
        ngram_range = ngram_range,
        max_features = n_terms)
    X = count_engine.fit_transform(DOC.doc_str)
    DTM = pd.DataFrame(X.toarray(), 
    columns=count_engine.get_feature_names_out(), 
    index=DOC.index)
    VOCAB = DTM.sum().to_frame('n')
    VOCAB.index.name = 'term_str'
    VOCAB['df'] = DTM.astype(bool).sum()
    VOCAB['dfidf'] = VOCAB.df * np.log2(len(DTM)/VOCAB.df)
    VOCAB['dp'] = VOCAB.df / len(DTM)
    VOCAB['di'] = np.log2(1/VOCAB.dp)
    VOCAB['dh'] = VOCAB.dp * VOCAB.di
    VOCAB['n_chars'] = VOCAB.apply(lambda x: len(x.name), 1)
    VOCAB['n_tokens'] = VOCAB.apply(lambda x: len(x.name.split()), 1)
    VOCAB.sort_index()


    #Hopefully this join works but I'm actually not too sure this will work
    
    return TFIDF,VOCAB,DTM

# I think I'm going to use this oneliner more.
# get_tfidf = lambda X, agg_func='sum': (X.T / X.T.agg(agg_func)).T * (np.log2(len(X)/X.astype('bool').sum()))
OHCO = ['book_id','chap_id', 'para_num', 'sent_num', 'token_num']

bags = dict(
    SENTS = OHCO[:4],
    PARAS = OHCO[:3],
    CHAPS = OHCO[:2],
    BOOKS = OHCO[:1]
)
tables = BOWtoTFIDF(BOW(CORPUS,'CHAPS'),'max', CORPUS, 2)
DTM = tables[2]
DFIDF = tables[1]
TFIDF = tables[0]

Tokenizing 5200 METAMORPHOSIS
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5200.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*[IVXLCM]+\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 5252 BLUMFELD
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5252.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone NOCHAPTERS
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 5353 THE BURROW
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5353.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone NOCHAPTERS
li

c:\Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/DS5001-2025-01-R/lessons/lib\textparser.py:132: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  div_lines = self.TOKENS[src_col].str.contains(div_pat, regex=True, case=True)


Done


## BOW

In [2]:
bow_df= BOW(CORPUS,'CHAPS')

In [3]:
TFIDF.T.columns

MultiIndex([( 5200,  1),
            ( 5200,  2),
            ( 5200,  3),
            ( 5252,  1),
            ( 5353,  1),
            ( 5454,  1),
            ( 5454,  2),
            ( 5555,  1),
            ( 5656,  1),
            ( 5757,  1),
            ( 5858,  1),
            ( 5959,  1),
            ( 6060,  1),
            ( 6161,  1),
            ( 6262,  1),
            ( 6262,  2),
            ( 6262,  3),
            ( 6262,  4),
            ( 6262,  5),
            ( 6262,  6),
            ( 6363,  1),
            ( 6464,  1),
            ( 6565,  1),
            ( 6666,  1),
            ( 6767,  1),
            ( 6969,  1),
            ( 6969,  2),
            ( 6969,  3),
            ( 6969,  4),
            ( 6969,  5),
            ( 6969,  6),
            ( 6969,  7),
            ( 6969,  8),
            ( 6969,  9),
            ( 6969, 10),
            ( 6969, 11),
            ( 6969, 12),
            ( 6969, 13),
            ( 6969, 14),
            ( 6969, 15),


In [4]:
# Step 1: Reset index so book_id and chap_id become columns
tfidf_reset = TFIDF.reset_index()

# Step 2: Melt all term columns into a long format
tfidf_long = tfidf_reset.melt(
    id_vars=['book_id', 'chap_id'],
    var_name='term_str',
    value_name='tfidf'
)

# Step 3: Merge with bag-of-words dataframe
bow_with_tfidf = bow_df.merge(
    tfidf_long,
    on=['book_id', 'chap_id', 'term_str'],
    how='left'
)

### BOW

In [5]:
#bow_with_tfidf.to_csv('BOW.csv')

In [6]:
bow_with_tfidf

,book_id,chap_id,term_str,n,tfidf
0,5200,1,a,108,0.000000
1,5200,1,abandoning,1,0.009230
2,5200,1,able,7,0.005868
3,5200,1,about,16,0.008273
4,5200,1,above,2,0.003155
...,...,...,...,...,...
80544,23532,10,you,55,0.068376
80545,23532,10,your,7,0.024125
80546,23532,10,youre,8,0.078501
80547,23532,10,yourself,2,0.018776


## DTM

In [7]:
DTM

term_str         abandoned  abandoning  abilities  ability  able  able make  \
book_id chap_id                                                               
5200    1                0           1          0        0     7          0   
        2                1           1          0        1    13          0   
        3                1           0          0        0     3          0   
5252    1                1           1          0        0     4          0   
5353    1                0           0          1        1     5          0   
...                    ...         ...        ...      ...   ...        ...   
23532   6                0           0          0        0     0          0   
        7                0           0          0        0     0          0   
        8                0           0          0        0     1          0   
        9                0           0          0        0     1          0   
        10               1           0          0        0     1          0   

term_str         abruptly  absence  absent  absolute  ...  ﬁre  ﬁre brigade  \
book_id chap_id                                       ...                     
5200    1               0        0       0         0  ...    0            0   
        2               0        0       0         0  ...    0            0   
        3               1        0       0         0  ...    0            0   
5252    1               0        1       0         0  ...    0            0   
5353    1               0        6       0         0  ...    0            0   
...                   ...      ...     ...       ...  ...  ...          ...   
23532   6               0        0       0         0  ...    0            0   
        7               0        0       0         0  ...    0            0   
        8               0        0       0         0  ...    0            0   
        9               0        0       0         0  ...    0            0   
        10              0        0       0         0  ...    0            0   

term_str         ﬁre engine  ﬁrmly  ﬁrst  ﬁrst time  ﬁst  ﬁt  ﬁxed  ﬂoor  
book_id chap_id                                                           
5200    1                 0      0     0          0    0   0     0     0  
        2                 0      0     0          0    0   0     0     0  
        3                 0      0     0          0    0   0     0     0  
5252    1                 0      0     0          0    0   0     0     0  
5353    1                 0      0     0          0    0   0     0     0  
...                     ...    ...   ...        ...  ...  ..   ...   ...  
23532   6                 0      0     0          0    0   0     0     0  
        7                 0      0     0          0    0   0     0     0  
        8                 0      0     0          0    0   0     0     0  
        9                 0      0     0          0    0   0     0     0  
        10                0      0     0          0    0   0     0     0  

[70 rows x 4000 columns]

In [8]:
#DTM.to_csv('DTM.csv')

## TFIDF

In [9]:
TFIDF

term_str           1   10   15    2   25    3  430  536    6    a  ...  ﬂoors  \
book_id chap_id                                                    ...          
5200    1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
5252    1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
5353    1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...   
23532   6        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        7        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        8        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        9        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        10       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   

term_str         ﬂopped  ﬂowered  ﬂowers  ﬂown  ﬂung  ﬂushed  ﬂuttered  \
book_id chap_id                                                          
5200    1           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        2           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        3           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
5252    1           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
5353    1           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
...                 ...      ...     ...   ...   ...     ...       ...   
23532   6           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        7           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        8           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        9           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        10          0.0      0.0     0.0   0.0   0.0     0.0       0.0   

term_str         ﬂuttering  ﬂying  
book_id chap_id                    
5200    1              0.0    0.0  
        2              0.0    0.0  
        3              0.0    0.0  
5252    1              0.0    0.0  
5353    1              0.0    0.0  
...                    ...    ...  
23532   6              0.0    0.0  
        7              0.0    0.0  
        8              0.0    0.0  
        9              0.0    0.0  
        10             0.0    0.0  

[70 rows x 14598 columns]

In [10]:
#TFIDF.to_csv('TFIDF.csv')

## Reduced and Normalized TFIDF_L2

In [11]:
tfidf_engine = TfidfTransformer(norm='l2', use_idf=True)
X1 = tfidf_engine.fit_transform(DTM)
L2 = pd.DataFrame(X1.toarray(), columns=DTM.columns, index=DTM.index)
L2

term_str         abandoned  abandoning  abilities  ability      able  \
book_id chap_id                                                        
5200    1         0.000000    0.008635   0.000000  0.00000  0.022226   
        2         0.005631    0.007872   0.000000  0.00680  0.037633   
        3         0.005539    0.000000   0.000000  0.00000  0.008543   
5252    1         0.003160    0.004418   0.000000  0.00000  0.006498   
5353    1         0.000000    0.000000   0.004867  0.00438  0.009324   
...                    ...         ...        ...      ...       ...   
23532   6         0.000000    0.000000   0.000000  0.00000  0.000000   
        7         0.000000    0.000000   0.000000  0.00000  0.000000   
        8         0.000000    0.000000   0.000000  0.00000  0.047635   
        9         0.000000    0.000000   0.000000  0.00000  0.091053   
        10        0.035651    0.000000   0.000000  0.00000  0.018329   

term_str         able make  abruptly   absence  absent  absolute  ...  ﬁre  \
book_id chap_id                                                   ...        
5200    1              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        2              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        3              0.0  0.007158  0.000000     0.0       0.0  ...  0.0   
5252    1              0.0  0.000000  0.003593     0.0       0.0  ...  0.0   
5353    1              0.0  0.000000  0.024749     0.0       0.0  ...  0.0   
...                    ...       ...       ...     ...       ...  ...  ...   
23532   6              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        7              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        8              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        9              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        10             0.0  0.000000  0.000000     0.0       0.0  ...  0.0   

term_str         ﬁre brigade  ﬁre engine  ﬁrmly  ﬁrst  ﬁrst time  ﬁst   ﬁt  \
book_id chap_id                                                              
5200    1                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        2                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        3                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
5252    1                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
5353    1                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
...                      ...         ...    ...   ...        ...  ...  ...   
23532   6                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        7                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        8                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        9                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        10               0.0         0.0    0.0   0.0        0.0  0.0  0.0   

term_str         ﬁxed  ﬂoor  
book_id chap_id              
5200    1         0.0   0.0  
        2         0.0   0.0  
        3         0.0   0.0  
5252    1         0.0   0.0  
5353    1         0.0   0.0  
...               ...   ...  
23532   6         0.0   0.0  
        7         0.0   0.0  
        8         0.0   0.0  
        9         0.0   0.0  
        10        0.0   0.0  

[70 rows x 4000 columns]

In [12]:
#L2.to_csv('L2.csv')

# PCA

In [13]:
LIB['work_type'] = LIB['title'].str.lower()
replacemap = {
    "amerika": "novel",
    "the castle": "novel",
    "the trial": "novel",
    
    "metamorphosis": "novella",
    "the burrow": "novella",
    "in the penal colony": "novella",
    "the hunger artist": "novella",
    
    "blumfeld": "short story",
    "josephine the songstress": "short story",
    "before the law": "short story",
    "up in the gallery": "short story",
    "the hunter gracchus": "short story",
    "the great wall of china": "short story",
    "a report for an academy": "short story",
    "the judgement": "short story",
    "the jackals and arabs": "short story",
    "a country doctor": "short story",
    "an imperial message": "short story",
    "meditation": "short story",
    
    "dearest father": "letter"
}
LIB['work_type'] = LIB['work_type'].replace(replacemap)
LIB

kafka_named_character_map = {
    "amerika": "named",
    "the castle": "unnamed",
    "the trial": "named",

    "metamorphosis": "named",
    "the burrow": "unnamed",
    "in the penal colony": "unnamed",
    "the hunger artist": "unnamed",

    "blumfeld": "named",
    "josephine the songstress": "named",
    "before the law": "unnamed",
    "up in the gallery": "unnamed",
    "the hunter gracchus": "named",
    "the great wall of china": "unnamed",
    "a report for an academy": "unnamed",
    "the judgement": "named",
    "the jackals and arabs": "unnamed",
    "a country doctor": "unnamed",
    "an imperial message": "unnamed",
    "meditation": "unnamed",

    "dearest father": "named"
}
LIB['protagonist_name'] = LIB['title'].str.lower()
LIB['protagonist_name'] = LIB['protagonist_name'].replace(kafka_named_character_map)
LIB['ending_type'] = LIB['title'].str.lower()

kafka_ending_map = {
    "amerika": "ambiguous",
    "the castle": "not happy",
    "the trial": "not happy",

    "metamorphosis": "not happy",
    "the burrow": "not happy",
    "in the penal colony": "not happy",
    "the hunger artist": "not happy",

    "blumfeld": "ambiguous",
    "josephine the songstress": "not happy",
    "before the law": "not happy",
    "up in the gallery": "ambiguous",
    "the hunter gracchus": "not happy",
    "the great wall of china": "ambiguous",
    "a report for an academy": "ambiguous",
    "the judgement": "not happy",
    "the jackals and arabs": "not happy",
    "a country doctor": "not happy",
    "an imperial message": "not happy",
    "meditation": "ambiguous",

    "dearest father": "not happy"
}
LIB['ending_type'] = LIB['ending_type'].replace(kafka_ending_map)
LIB

,source_file_path,author,title,chap_regex,work_type,protagonist_name,ending_type
book_id,,,,,,,
5200,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",METAMORPHOSIS,^\s*[IVXLCM]+\s*$,novella,named,not happy
5252,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",BLUMFELD,NOCHAPTERS,short story,named,ambiguous
5353,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",THE BURROW,NOCHAPTERS,novella,unnamed,not happy
5454,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",JOSEPHINE THE SONGSTRESS,NOCHAPTERS,short story,named,not happy
5555,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",BEFORE THE LAW,NOCHAPTERS,short story,unnamed,not happy
5656,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",UP IN THE GALLERY,NOCHAPTERS,short story,unnamed,ambiguous
5757,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",THE HUNTER GRACCHUS,NOCHAPTERS,short story,named,not happy
5858,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",THE GREAT WALL OF CHINA,NOCHAPTERS,short story,unnamed,ambiguous
5959,/Users/muhur/OneDrive/Desktop/Muhurto/Data Sci...,"KAFKA, FRANZ",A REPORT FOR AN ACADEMY,NOCHAPTERS,short story,unnamed,ambiguous


In [14]:
import pandas as pd
import numpy as np
from scipy.linalg import norm
import plotly_express as px
import seaborn as sns
from sklearn.decomposition import PCA
sns.set(style='ticks')
OHCO = ['book_id', 'chap_id']
# Added after class
norm_docs = True # This has the effect of exaggerating variance when False
center_term_vectors = True # This has the effect of demoting authorship when False
colors = "YlGnBu" 
# colors = "Spectral"




pca_engine = PCA(n_components=10)
DCM = pd.DataFrame(pca_engine.fit_transform(L2.fillna(0)), index=L2.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM = DCM.join(LIB[['author','work_type']], on='book_id') #the lavel i will use is work_type
def vis_pcs(M, a, b, label='author', hover_name='author', symbol=None, size=None):
    return px.scatter(M, f"PC{a}", f"PC{b}", color=label, hover_name=hover_name, 
                     symbol=symbol, size=size,
                     marginal_x='box', height=800)

def vis_loadings(a=0, b=1, hover_name='term_str'):
    return px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      text='term_str', hover_name='term_str',
                      size='dfidf', color='max_pos_group', 
                      marginal_x='box', height=800)
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index = L2.columns
LOADINGS.index.name = 'term_str'
LOADINGS = LOADINGS.join(VOCAB)
vis_pcs(DCM, 0, 1, label='work_type', hover_name= 'work_type')

In [15]:
import pandas as pd
import numpy as np
from scipy.linalg import norm
import plotly_express as px
import seaborn as sns
from sklearn.decomposition import PCA
sns.set(style='ticks')
OHCO = ['book_id', 'chap_id']
# Added after class
norm_docs = True # This has the effect of exaggerating variance when False
center_term_vectors = True # This has the effect of demoting authorship when False
colors = "YlGnBu" 
# colors = "Spectral"




pca_engine = PCA(n_components=10)
DCM = pd.DataFrame(pca_engine.fit_transform(L2.fillna(0)), index=L2.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM = DCM.join(LIB[['author','protagonist_name']], on='book_id') #the lavel i will use is work_type
def vis_pcs(M, a, b, label='author', hover_name='author', symbol=None, size=None):
    return px.scatter(M, f"PC{a}", f"PC{b}", color=label, hover_name=hover_name, 
                     symbol=symbol, size=size,
                     marginal_x='box', height=800)

def vis_loadings(a=0, b=1, hover_name='term_str'):
    return px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      text='term_str', hover_name='term_str',
                      size='dfidf', color='max_pos_group', 
                      marginal_x='box', height=800)
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index = L2.columns
LOADINGS.index.name = 'term_str'
LOADINGS = LOADINGS.join(VOCAB)
vis_pcs(DCM, 0, 1, label='protagonist_name', hover_name= 'protagonist_name')

In [16]:
import pandas as pd
import numpy as np
from scipy.linalg import norm
import plotly_express as px
import seaborn as sns
from sklearn.decomposition import PCA
sns.set(style='ticks')
OHCO = ['book_id', 'chap_id']
# Added after class
norm_docs = True # This has the effect of exaggerating variance when False
center_term_vectors = True # This has the effect of demoting authorship when False
colors = "YlGnBu" 
# colors = "Spectral"




pca_engine = PCA(n_components=10)
DCM = pd.DataFrame(pca_engine.fit_transform(L2.fillna(0)), index=L2.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM = DCM.join(LIB[['author','ending_type']], on='book_id') #the lavel i will use is work_type
def vis_pcs(M, a, b, label='author', hover_name='author', symbol=None, size=None):
    return px.scatter(M, f"PC{a}", f"PC{b}", color=label, hover_name=hover_name, 
                     symbol=symbol, size=size,
                     marginal_x='box', height=800)

def vis_loadings(a=0, b=1, hover_name='term_str'):
    return px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      text='term_str', hover_name='term_str',
                      size='dfidf', color='max_pos_group', 
                      marginal_x='box', height=800)
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index = L2.columns
LOADINGS.index.name = 'term_str'
LOADINGS = LOADINGS.join(VOCAB)
vis_pcs(DCM, 0, 1, label='ending_type', hover_name= 'ending_type')

In [17]:
import pandas as pd
import numpy as np
from scipy.linalg import norm
import plotly_express as px
import seaborn as sns
from sklearn.decomposition import PCA
sns.set(style='ticks')
OHCO = ['book_id', 'chap_id']
# Added after class
norm_docs = True # This has the effect of exaggerating variance when False
center_term_vectors = True # This has the effect of demoting authorship when False
colors = "YlGnBu" 
# colors = "Spectral"




pca_engine = PCA(n_components=10)
DCM = pd.DataFrame(pca_engine.fit_transform(L2.fillna(0)), index=L2.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM = DCM.join(LIB[['author','title']], on='book_id') #the lavel i will use is work_type
def vis_pcs(M, a, b, label='author', hover_name='author', symbol=None, size=None):
    return px.scatter(M, f"PC{a}", f"PC{b}", color=label, hover_name=hover_name, 
                     symbol=symbol, size=size,
                     marginal_x='box', height=800)

def vis_loadings(a=0, b=1, hover_name='term_str'):
    return px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      text='term_str', hover_name='term_str',
                      size='dfidf', color='max_pos_group', 
                      marginal_x='box', height=800)
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index = L2.columns
LOADINGS.index.name = 'term_str'
LOADINGS = LOADINGS.join(VOCAB)
vis_pcs(DCM, 0, 1, label='title', hover_name= 'title')

In [18]:
vis_pcs(DCM, 2, 3, label='title', hover_name= 'title')

## PCA COMPONENTS

In [27]:
LOADINGS['PC0'].sort_values(ascending=False).head(10)

term_str
karl            0.110113
uncle           0.021334
delamarche      0.018319
didn            0.018258
gregor          0.017940
pollunder       0.017875
said karl       0.017219
mr pollunder    0.016232
robinson        0.015669
stoker          0.014555
Name: PC0, dtype: float64

In [28]:
LOADINGS['PC1'].sort_values(ascending=True).head(10)

term_str
gregor       -0.034040
horse        -0.018353
gregors      -0.016428
sister       -0.014823
father       -0.012899
folk         -0.011762
horses       -0.011278
josephine    -0.010986
gatekeeper   -0.009761
beautiful    -0.008965
Name: PC1, dtype: float64

In [41]:
COMPONENTS = pd.DataFrame(pca_engine.components_)
COMPONENTS
#COMPONENTS.to_csv('COMPONENTS.csv')

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,0.002523,-0.001021,-0.000979,0.004006,0.009875,-0.001858,0.000409,-0.000376,0.001700,0.001568,...,-0.027285,-0.011090,-0.009217,-0.012083,-0.089827,-0.006937,-0.008719,-0.007632,-0.009848,-0.023944
1,-0.001528,-0.000104,-0.000766,-0.008258,-0.010412,0.001288,0.002130,0.003624,0.000446,-0.001115,...,0.001615,-0.000997,0.000691,0.004967,0.036579,0.002766,0.004756,0.002369,0.004381,0.012749
2,-0.005251,0.002003,0.004161,-0.003150,-0.003729,0.002643,-0.001041,0.008035,0.000499,-0.002344,...,0.081112,0.044808,0.030390,0.004928,0.053175,0.003121,-0.003693,0.004338,0.005178,-0.010491
3,0.004873,0.004823,-0.000500,-0.009317,-0.002135,0.001242,0.003666,0.000511,0.001871,0.000126,...,-0.004467,-0.002627,-0.003975,0.000770,-0.007477,0.000399,0.000573,0.000889,0.000063,0.003128
4,-0.007745,0.003352,-0.001595,0.006060,0.004752,-0.009343,-0.000436,0.000729,-0.000105,-0.007648,...,0.004730,0.000415,0.003712,0.004102,0.024257,0.001580,0.005787,-0.000342,0.001861,0.014278
5,-0.004925,-0.000125,0.001059,0.034055,0.051932,-0.001020,0.001598,0.003806,0.001038,0.010998,...,0.009898,0.004587,0.001686,-0.000416,-0.007373,0.000139,0.000772,-0.000484,-0.003436,0.000395
6,0.000440,0.001298,-0.007423,0.009684,0.006820,0.002237,0.001946,-0.008306,-0.004697,-0.001279,...,-0.113558,-0.056279,-0.043553,-0.008585,-0.008269,-0.007231,-0.004198,-0.003678,-0.006556,-0.022447
7,-0.009598,0.004763,-0.006542,-0.000557,0.023430,0.001624,-0.000976,-0.006303,0.002331,0.002049,...,-0.013832,-0.007699,-0.001557,-0.006229,-0.004355,-0.002707,-0.001750,-0.004783,0.003716,0.003370
8,0.006365,-0.002992,0.000151,0.008381,0.036127,-0.000474,-0.000186,-0.002687,0.000341,-0.000223,...,0.042305,0.022456,0.017100,0.005450,0.018787,0.002831,-0.001495,0.003253,-0.000521,0.000044
9,-0.006092,-0.002855,-0.000881,-0.010845,-0.019865,-0.002823,0.007396,0.004948,0.002853,0.005058,...,0.034549,0.020013,0.017038,0.002010,0.007928,0.002643,-0.004711,-0.002118,-0.001326,-0.013982


In [40]:
DCM
#DCM.to_csv('DCM.csv')

PC0       PC1       PC2       PC3       PC4       PC5  \
book_id chap_id                                                               
5200    1        0.141323 -0.255321  0.099228  0.677108  0.196797  0.029879   
        2        0.176970 -0.281937  0.131181  0.733911  0.286330  0.014847   
        3        0.168403 -0.256803  0.106427  0.686433  0.250481  0.003529   
5252    1        0.135361 -0.138505 -0.021799 -0.101060  0.100858  0.026625   
5353    1        0.100131 -0.141379  0.065595 -0.104005  0.040792  0.110012   
...                   ...       ...       ...       ...       ...       ...   
23532   6        0.168615 -0.179751 -0.015748 -0.177604  0.181144 -0.126465   
        7        0.166980 -0.177764 -0.039697 -0.170304  0.141929 -0.138057   
        8        0.181401 -0.215050 -0.030430 -0.205805  0.141702 -0.202402   
        9        0.189385 -0.207692 -0.060927 -0.201679  0.208135 -0.274031   
        10       0.008088 -0.076714 -0.125617  0.050308 -0.177711 -0.027143   

                      PC6       PC7       PC8       PC9        author  \
book_id chap_id                                                         
5200    1        0.118829  0.048355 -0.019312 -0.017311  KAFKA, FRANZ   
        2        0.104435  0.037605 -0.015908 -0.015712  KAFKA, FRANZ   
        3        0.097194  0.044844  0.009694 -0.017391  KAFKA, FRANZ   
5252    1        0.076850  0.085376 -0.174538  0.024125  KAFKA, FRANZ   
5353    1       -0.010286 -0.145312 -0.208212 -0.024751  KAFKA, FRANZ   
...                   ...       ...       ...       ...           ...   
23532   6       -0.070660 -0.152791 -0.070831  0.152756  KAFKA, FRANZ   
        7        0.000898 -0.117480 -0.018872  0.167170  KAFKA, FRANZ   
        8        0.011512  0.129575  0.276258 -0.001765  KAFKA, FRANZ   
        9        0.007774  0.263470  0.303637  0.084852  KAFKA, FRANZ   
        10      -0.006566 -0.114776  0.115790 -0.081661  KAFKA, FRANZ   

                         title  
book_id chap_id                 
5200    1        METAMORPHOSIS  
        2        METAMORPHOSIS  
        3        METAMORPHOSIS  
5252    1             BLUMFELD  
5353    1           THE BURROW  
...                        ...  
23532   6           MEDITATION  
        7           MEDITATION  
        8           MEDITATION  
        9           MEDITATION  
        10          MEDITATION  

[70 rows x 12 columns]

In [44]:
LOADINGS
#LOADINGS.to_csv('LOADINGS.csv')

,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,max_pos,max_pos_group,n_pos_group,cat_pos_group,n_pos,cat_pos,stop,porter_stem,stem_snowball,stem_lancaster
term_str,,,,,,,,,,,,,,,,,,,,,
abandoned,0.000621,-0.000326,-0.000945,0.000852,-0.001281,-0.000727,0.000061,-0.001325,0.000860,-0.000813,...,VBN,VB,4.0,"{RP, VB, NN, JJ}",5.0,"{VBN, RP, NN, VBD, JJ}",0.0,abandon,abandon,abandon
abandoning,-0.000251,-0.000022,0.000360,0.000843,0.000554,-0.000018,0.000181,0.000658,-0.000404,-0.000381,...,VBG,VB,1.0,{VB},1.0,{VBG},0.0,abandon,abandon,abandon
abilities,-0.000241,-0.000163,0.000749,-0.000087,-0.000264,0.000156,-0.001037,-0.000903,0.000020,-0.000118,...,NNS,NN,1.0,{NN},2.0,"{NNS, NN}",0.0,abil,abil,abl
ability,0.000986,-0.001762,-0.000567,-0.001629,0.001002,0.005027,0.001352,-0.000077,0.001132,-0.001448,...,NN,NN,1.0,{NN},1.0,{NN},0.0,abil,abil,abl
able,0.002430,-0.002222,-0.000671,-0.000373,0.000786,0.007666,0.000952,0.003235,0.004878,-0.002652,...,JJ,JJ,4.0,"{VB, RB, NN, JJ}",5.0,"{RB, VBN, NN, VBP, JJ}",0.0,abl,abl,abl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ﬁrst time,-0.001707,0.000590,0.000562,0.000070,0.000261,0.000021,-0.001010,-0.000374,0.000382,0.000353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ﬁst,-0.002145,0.001015,-0.000665,0.000100,0.000957,0.000114,-0.000586,-0.000242,-0.000202,-0.000629,...,NN,NN,2.0,"{FW, NN}",2.0,"{FW, NN}",0.0,ﬁst,ﬁst,ﬁst
ﬁt,-0.001878,0.000506,0.000780,0.000156,-0.000057,-0.000071,-0.000514,-0.000660,0.000439,-0.000283,...,NN,NN,4.0,"{VB, CD, NN, JJ}",6.0,"{CD, NN, VBP, VB, NNS, JJ}",0.0,ﬁt,ﬁt,ﬁt
